In [4]:
import google.generativeai as genai

In [ ]:
import google.generativeai as genai

genai.configure(api_key="")

model = genai.GenerativeModel("gemini-2.5-flash-preview-04-17")  # Use the correct model name
response = model.generate_content("Explain how AI works in a few words")

print(response.text)

It learns patterns from data.


In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

# --- Configuration ---
PROJECT_ID = "gen-lang-client-0061421706"  # Replace with your actual project ID
LOCATION = "us-central1"  # Choose a region where Gemini 1.5 Flash is available (e.g., us-central1)
MODEL_NAME = "gemini-1.5-flash-001"  # Or "gemini-1.5-flash" for the latest stable alias

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Load the model
model = GenerativeModel(MODEL_NAME)

print(f"Vertex AI initialized for project: {PROJECT_ID} in location: {LOCATION}")
print(f"Model loaded: {MODEL_NAME}")

In [ ]:
# Text generation
prompt = "Explain the concept of quantum entanglement in simple terms."
response = model.generate_content(prompt)

print("\n--- Text Generation Response ---")
print(response.text)

In [ ]:
from google import genai
from google.genai import types
import base64


def generate():
    client = genai.Client(
        vertexai=True,
        project="gen-lang-client-0061421706",
        location="global",
    )

    model = "gemini-2.5-flash-preview-05-20"
    contents = [types.Content(role="user", parts=[types.Part(text="Explain how AI works in a few words")])]

    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        seed=0,
        max_output_tokens=300,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"
            ),
            types.SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"
            ),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")


In [13]:
generate()

AI learns from data to find patterns and make predictions or decisions.